# Guess Core and use other methods to interpolate

In [ ]:
import os, sys
import pickle
from scf_guess_tools import  Backend, load, calculate, guess
import numpy as np
import matplotlib.pyplot as plt
from BlockMatrix import BlockMatrix
from tensorflow.keras.callbacks import TensorBoard
import datetime

#! Only if there are no cudo CPUs in the system!
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"


### 1b) Try on other dataset!

In [ ]:
import sys, os
from glob import glob
sys.path.append("..")
from utils import  plot_mat_comp, flatten_triang_batch, flatten_triang, get_overlap, load_mol, unflatten_triang, density_from_fock, perform_calculation
scripts_path = "../../scripts"
if scripts_path not in sys.path:
    sys.path.append(scripts_path)
from to_cache import density_fock_overlap


In [ ]:
train_test_seed = 42
source_path = '../../datasets/QM9/xyz_c7h10o2/'
all_file_paths = glob(os.path.join(source_path, '*.xyz'))
n_elec = 19
len(all_file_paths)


In [ ]:
def load_cached(file_paths, cache_path, basis, guess="minao", method="dft", functional="b3lypg", backend="pyscf"):
    error_list = []
    error_files = []
    focks = []
    overlaps = []
    used_files = []
    reference_densities = []
    for file in file_paths:
        mol_name = os.path.basename(file).strip()
        # print(mol_name)
        try: 
            ret = density_fock_overlap(filepath = file,
                                filename = mol_name,
                                method = method,
                                basis = None,
                                functional = functional,
                                guess = guess,
                                backend = backend,
                                cache = cache_path)
            # print(f"Using: file={file} - mol_name={mol_name} - basis={None} - guess={guess} - method={method} - functional={functional}")
        except Exception as e: 
            error_list.append(e)
            error_files.append(mol_name)
            print(f"File {mol_name} error - skipping")
            continue
        if any([r == None for r in ret]): 
            print(f"File {mol_name} bad - skipping")
            continue
        focks.append(ret[1].numpy)
        used_files.append(file)
        reference_densities.append(ret[0].numpy)
        overlaps.append(ret[2].numpy)
    print(f"Got data for: {len(focks)} - bad / no ret: {len(file_paths) - len(focks) - len(error_list)} - errors: {len(error_list)}")
    print(error_files[:5])
    return focks, reference_densities, overlaps, used_files


In [ ]:
ret = load_cached(all_file_paths, "../../datasets/QM9/out/c7h10o2_b3lypg_6-31G(2df,p)/pyscf", basis="6-31g_2df_p_custom_nwchem.gbs")

In [ ]:
from sklearn.model_selection import train_test_split
train_test_seed = 42
all_data = [(fock, ref_density, overlap, file) for fock, ref_density, overlap, file in zip(*ret)]
print(len(all_data))
train_data, test_data = train_test_split(all_data, test_size=0.2, random_state=train_test_seed) 

In [ ]:
mat_dim = train_data[0][0].shape
mat_dim

In [ ]:
train_overlap_X = np.array([flatten_triang(data[2]) for data in train_data])
# train_fock_Y = np.array([data[0] for data in train_data])
train_fock_diag = np.array([np.diag(data[0]) for data in train_data])

test_overlap_X = np.array([flatten_triang(data[2]) for data in test_data])
# test_fock_Y = np.array([data[0] for data in test_data])
test_fock_diag = np.array([np.diag(data[0]) for data in test_data])
train_overlap_X.shape, test_overlap_X.shape

In [ ]:
import pickle as pk
with open("data/train_overlap_X.pk", "wb") as f:
    pk.dump(train_overlap_X, f)
with open("data/train_fock_diag.pk", "wb") as f:
    pk.dump(train_fock_diag, f)
with open("data/test_overlap_X.pk", "wb") as f:
    pk.dump(test_overlap_X, f)
with open("data/test_fock_diag.pk", "wb") as f:
    pk.dump(test_fock_diag, f)

In [ ]:
# load data
import pickle as pk
with open("data/train_overlap_X.pk", "rb") as f:
    train_overlap_X = pk.load(f)
with open("data/train_fock_diag.pk", "rb") as f:
    train_fock_diag = pk.load(f)
with open("data/test_overlap_X.pk", "rb") as f:
    test_overlap_X = pk.load(f)
with open("data/test_fock_diag.pk", "rb") as f:
    test_fock_diag = pk.load(f)

In [ ]:
custom_631g_basis = "../../scripts/6-31g_2df_p_custom_nwchem.gbs"
example_mol_filepath = os.path.basename(train_data[0][3]).strip()
example_mol = load_mol(train_data[0][3], custom_631g_basis, Backend.PY)
example_ovlp = unflatten_triang(train_overlap_X[0], mat_dim[0])
example_overlap = BlockMatrix(example_mol, example_ovlp)
example_overlap.plot_blocks_by_type("all", labels="atoms", figsize=(10, 10), imshow_args={"cmap": "RdBu"})

let's try our luck! - no rescaling for now 

In [ ]:
import tensorflow as tf
in_dim = train_overlap_X.shape[1]
out_dim = train_fock_diag.shape[1]
in_dim, out_dim

In [ ]:
tf.random.set_seed(42)
flattened_dim = train_overlap_X.shape[1]
diag_dim = train_fock_diag.shape[1]
inputs_orig = tf.keras.Input(shape=(flattened_dim,))
x = inputs_orig
for neurons in [1024,512,256]:
    x = tf.keras.layers.Dense(neurons, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(diag_dim)(x)

basic_model = tf.keras.Model(inputs=inputs_orig, outputs=outputs)
basic_model.compile(optimizer='adam', loss="mae", metrics=["mse"])

In [ ]:
basic_model.summary()

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint("models/basic_model_6_31G.h5", save_best_only=True)
basic_hist = basic_model.fit(train_overlap_X, train_fock_diag,
                            validation_data=(test_overlap_X, test_fock_diag),
                            epochs=100,
                            batch_size=32,
                            callbacks=[tensorboard_callback, checkpoint_callback],
                            verbose=1)
# Plot the training and validation loss
plt.plot(basic_hist.history['loss'], label='train')
plt.plot(basic_hist.history['val_loss'], label='validation')
plt.yscale('log')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('mae')
plt.title('Model loss')

In [ ]:
os.makedirs("models", exist_ok=True)
if os.path.exists("models/basic_model_6_31G.keras") and input("Overwrite existing model? (y/n)") == "y":
    os.remove("models/basic_model_6_31G.keras")
    basic_model.save("models/basic_model_6_31G.keras")
    print("Model saved")
else:
    basic_model.save("models/basic_model_6_31G.keras")
    print("Model saved")

In [ ]:
# load model
loaded_model = tf.keras.models.load_model("models/basic_model_6_31G.keras")
basic_model = loaded_model

In [ ]:
def reconstruct_Fock(diag, ovlp, K = 1.75): 
    """Take diagonal and reconstruct the Fock matrix using GWH
    """
    mat_dim = diag.shape[0]
    out = np.zeros((mat_dim, mat_dim))
    for i in range(mat_dim):
        for j in range(mat_dim):
            if i == j:
                out[i, j] = diag[i]
            else:
                out[i, j] = K * ovlp[i, j] * (diag[i] + diag[j]) / 2
    return out

In [ ]:
# predict: 
test_pred_fock_diag = basic_model.predict(test_overlap_X)

In [ ]:
# example comparison: 
rand_test_sample = np.random.randint(0, len(test_pred_fock_diag))
pred_fock_example = reconstruct_Fock(test_pred_fock_diag[rand_test_sample], unflatten_triang(test_overlap_X[rand_test_sample], mat_dim[0]))
ground_truth_fock_example = test_data[rand_test_sample][0]
pred_density_example = density_from_fock(pred_fock_example, unflatten_triang(test_overlap_X[rand_test_sample], mat_dim[0]), mat_dim[0])
# compare with hückel and minao
test_mol = load(test_data[rand_test_sample][3], symmetry=False, basis = custom_631g_basis, backend=Backend.PY)
minao_guess = guess(test_mol, method="hf", basis=custom_631g_basis, scheme="minao")
hueckel_guess = guess(test_mol, method="hf", basis=custom_631g_basis, scheme="huckel")
plot_mat_comp(ground_truth_fock_example, pred_fock_example, title="Fock matrix prediction Basic NN", vmax=0.5)
plot_mat_comp(ground_truth_fock_example, minao_guess.fock().numpy, title="Fock matrix prediction MINAO", vmax=0.5)
plot_mat_comp(ground_truth_fock_example, hueckel_guess.fock().numpy, title="Fock matrix prediction Hückel", vmax=0.5)

In [ ]:
# test sim
minao_res = calculate(test_mol, method="dft", functional="b3lypg", basis=custom_631g_basis, guess="minao")

In [ ]:
huckel_res = calculate(test_mol, method="dft", functional="b3lypg", basis=custom_631g_basis, guess="huckel")

In [ ]:
nn_res = perform_calculation(test_data[rand_test_sample][3], pred_density_example, method="dft", functional="b3lypg", basis_set=custom_631g_basis)

In [ ]:
print(minao_res.native.cycles)
print(huckel_res.native.cycles)
print(nn_res["cycles"])